# Preprocessing data (used for 2017-2022)

In [1]:
import pandas as pd

Read in all the data, using the year of the data as a key

In [2]:
dfs = {}
for i in range(2016, 2023):
    directory = f"Data//Headways_{i}"
    df = pd.read_csv(directory + ".csv")
    dfs[i] = df

In [3]:
for df in dfs:
    print(dfs[df].head())

           service_date route_id  direction_id  stop_id  start_time_sec  \
0  1/1/2016 12:00:00 AM     Blue             0    70041           24710   
1  1/1/2016 12:00:00 AM     Blue             0    70041           25443   
2  1/1/2016 12:00:00 AM     Blue             0    70041           26332   
3  1/1/2016 12:00:00 AM     Blue             0    70041           26972   
4  1/1/2016 12:00:00 AM     Blue             0    70041           27740   

   end_time_sec  headway_time_sec destination  ObjectId  
0         25443               733     Bowdoin         1  
1         26332               889     Bowdoin         2  
2         26972               640     Bowdoin         3  
3         27740               768     Bowdoin         4  
4         28376               636     Bowdoin         5  
           service_date route_id  direction_id  stop_id  start_time_sec  \
0  1/1/2017 12:00:00 AM     Blue             0    70039           82949   
1  1/1/2017 12:00:00 AM     Blue             0    7

Group data by stop, direction, and destination.

In [4]:
for df in dfs:
    df_grouped = dfs[df].groupby(['stop_id', 'direction_id', 'destination']).agg({
        'headway_time_sec': 'mean'
    }).reset_index()
    dfs[df] = df_grouped
    print(df_grouped.head())

   stop_id  direction_id   destination  headway_time_sec
0    70001             1     Oak Grove        499.692239
1    70002             0  Forest Hills        509.656499
2    70003             1     Oak Grove        512.070998
3    70004             0  Forest Hills        506.538108
4    70005             1     Oak Grove        512.941418
   stop_id  direction_id   destination  headway_time_sec
0    70001             1     Oak Grove        505.786136
1    70002             0  Forest Hills        505.573351
2    70003             1     Oak Grove        507.114465
3    70004             0  Forest Hills        505.320893
4    70005             1     Oak Grove        509.227303
   stop_id  direction_id   destination  headway_time_sec
0    70001             0  Forest Hills       1056.137931
1    70001             1     Oak Grove        512.213911
2    70002             0  Forest Hills        519.718360
3    70003             1     Oak Grove        514.117837
4    70004             0  Fores

Check for null values

In [5]:
for df in dfs:
    print(dfs[df].isnull().sum())

stop_id             0
direction_id        0
destination         0
headway_time_sec    0
dtype: int64
stop_id             0
direction_id        0
destination         0
headway_time_sec    0
dtype: int64
stop_id             0
direction_id        0
destination         0
headway_time_sec    0
dtype: int64
stop_id             0
direction_id        0
destination         0
headway_time_sec    0
dtype: int64
stop_id             0
direction_id        0
destination         0
headway_time_sec    0
dtype: int64
stop_id             0
direction_id        0
destination         0
headway_time_sec    0
dtype: int64
stop_id             0
direction_id        0
destination         0
headway_time_sec    0
dtype: int64


# Creating stop_name column for data

Using MBTA's GTFS stop.txt file

In [6]:
# Read the CSV file into a DataFrame
df = pd.read_csv('stops.csv')

# Ensure that 'stop_id' is numeric and drop rows where 'stop_id' is not a valid number
df['stop_id'] = pd.to_numeric(df['stop_id'], errors='coerce')

# Drop rows where 'stop_id' is NaN (i.e., not numeric)
df = df.dropna(subset=['stop_id'])

# Convert 'stop_id' to integers
df['stop_id'] = df['stop_id'].astype(int)

# Create a dictionary with stop_id as keys and stop_name as values
stop_dict = pd.Series(df.stop_name.values, index=df.stop_id).to_dict()
stop_desc_dict = pd.Series(df.stop_desc.values, index=df.stop_id).to_dict()

# Print the resulting dictionary
print(stop_dict)
print(stop_desc_dict)

{1: 'Washington St opp Ruggles St', 10: 'Theo Glynn Way @ Newmarket Sq', 10000: 'Tremont St opp Temple Pl', 10003: 'Albany St opp Randall St', 10005: 'Albany St opp E Concord St', 10006: 'Albany St opp E Newton St', 10007: '540 Albany St opp Wareham St', 10008: 'Albany St opp Randolph St', 10009: 'High St @ Duck Lane', 10010: 'Southampton St @ Newmarket St', 10011: 'Albany St @ Randolph St', 10012: 'Albany St @ Wareham St', 10013: 'Albany St @ E Dedham St', 10014: 'Albany St opp Boston Medical', 10015: 'Harrison Ave @ E Newton St', 10016: 'TradeCenter 128', 10031: 'E 1st St @ P St', 10032: 'E 1st St opp O St', 10033: 'E 1st St @ O St', 101: 'Massachusetts Ave @ Sidney St', 10100: 'Albany St @ Randall St', 10101: 'Melnea Cass Blvd @ Harrison Ave', 102: 'Massachusetts Ave @ Prospect St', 1026: 'Chestnut Hill Ave @ Veronica Smith Ctr', 1027: 'Chestnut Hill Ave @ Englewood Ave', 1029: 'Chestnut Hill Ave @ Commonwealth Ave', 1030: 'Chestnut Hill Ave @ Strathmore Rd', 1032: 'Chestnut Hill Av

Check for stop_ids that are not in stops.csv

In [7]:
unassigned_stops = []
for i in dfs:
    df = dfs[i]
    # Create the 'stop_name' column by mapping 'stop_id' using the stop_id_to_name dictionary
    df['stop_name'] = df['stop_id'].map(stop_dict)
    df['stop_desc'] = df['stop_id'].map(stop_desc_dict)
    print(df.head())
    # Filter rows where 'stop_name' is blank (either NaN or empty string)
    blank_stop_names = df[df['stop_name'].isna() | (df['stop_name'] == '')]

    # Get unique stop_ids from those rows
    unassigned_stops.extend(blank_stop_names['stop_id'].tolist())

# Print the unique stop_ids
unique_unassigned_stops = list(set(unassigned_stops))
print(unique_unassigned_stops)


   stop_id  direction_id   destination  headway_time_sec     stop_name  \
0    70001             1     Oak Grove        499.692239  Forest Hills   
1    70002             0  Forest Hills        509.656499  Green Street   
2    70003             1     Oak Grove        512.070998  Green Street   
3    70004             0  Forest Hills        506.538108   Stony Brook   
4    70005             1     Oak Grove        512.941418   Stony Brook   

                                   stop_desc  
0                 Forest Hills - Orange Line  
1  Green Street - Orange Line - Forest Hills  
2     Green Street - Orange Line - Oak Grove  
3   Stony Brook - Orange Line - Forest Hills  
4      Stony Brook - Orange Line - Oak Grove  
   stop_id  direction_id   destination  headway_time_sec     stop_name  \
0    70001             1     Oak Grove        505.786136  Forest Hills   
1    70002             0  Forest Hills        505.573351  Green Street   
2    70003             1     Oak Grove        507.1

Check that all MBTA stops are included in the datasets

In [8]:
red_line = ['Alewife', 'Davis', 'Porter', 'Harvard', 'Central', 'Kendall/MIT', 'Charles/MGH', 'Park Street', 'Downtown Crossing', 'South Station', 'Broadway', 'Andrew', 'JFK/UMass', 'Savin Hill', 'Fields Corner', 'Shawmut', 'Ashmont', 'Cedar Grove', 'Butler', 'Milton', 'Central Avenue', 'Valley Road', 'Capen Street', 'Mattapan', 'North Quincy', 'Wollaston', 'Quincy Center', 'Quincy Adams', 'Braintree']
green_line = ['Medford/Tufts', 'Ball Square', 'Magoun Square', 'Gilman Square', 'East Somerville', 'Union Square', 'Lechmere', 'Science Park/West End', 'North Station', 'Haymarket', 'Government Center', 'Park Street', 'Boylston', 'Arlington', 'Copley', 'Hynes Convention Center', 'Kenmore', 'Blandford Street', 'Boston University East', 'Boston University Central', 'Amory Street', 'Babcock Street', 'Packard\'s Corner', 'Harvard Avenue', 'Griggs Street', 'Allston Street', 'Warren Street', 'Washington Street', 'Sutherland Road', 'Chiswick Road', 'Chestnut Hill Avenue', 'South Street', 'Boston College', "Saint Mary's Street", 'Hawes Street', 'Kent Street', 'Saint Paul Street', 'Coolidge Corner', 'Summit Avenue', 'Brandon Hall', 'Fairbanks Street', 'Washington Square', 'Tappan Street', 'Dean Road', 'Englewood Avenue', 'Cleveland Circle', 'Fenway', 'Longwood', 'Brookline Village', 'Brookline Hills', 'Beaconsfield', 'Reservoir', 'Chestnut Hill', 'Newton Centre', 'Newton Highlands', 'Eliot', 'Waban', 'Woodland', 'Riverside', 'Prudential', 'Symphony', 'Northeastern University', 'Museum of Fine Arts', 'Longwood Medical Area', 'Brigham Circle', 'Fenwood Road', 'Mission Park', 'Riverway', 'Back of the Hill', 'Heath Street']
blue_line = ['Bowdoin', 'Government Center', 'State', 'Aquarium', 'Maverick', 'Airport', 'Wood Island', 'Orient Heights', 'Suffolk Downs', 'Beachmont', 'Revere Beach', 'Wonderland']
orange_line = ['Oak Grove', 'Malden Center', 'Wellington', 'Assembly', 'Sullivan Square', 'Community College', 'North Station', 'Haymarket', 'State', 'Downtown Crossing', 'Chinatown', 'Tufts Medical Center', 'Back Bay', 'Massachusetts Avenue', 'Ruggles', 'Roxbury Crossing', 'Jackson Square', 'Stony Brook', 'Green Street', 'Forest Hills']

In [9]:
for i in dfs:
    df = dfs[i]
    print(f"Unique stops in the {i} data: {df['stop_name'].unique()}")
    print()

Unique stops in the 2016 data: ['Forest Hills' 'Green Street' 'Stony Brook' 'Jackson Square'
 'Roxbury Crossing' 'Ruggles' 'Massachusetts Avenue' 'Back Bay'
 'Tufts Medical Center' 'Chinatown' 'Downtown Crossing' 'State'
 'Haymarket' 'North Station' 'Community College' 'Sullivan Square'
 'Wellington' 'Malden Center' 'Oak Grove' 'Bowdoin' 'Government Center'
 'Aquarium' 'Maverick' 'Airport' 'Wood Island' 'Orient Heights'
 'Suffolk Downs' 'Beachmont' 'Revere Beach' 'Wonderland' 'Alewife' 'Davis'
 'Porter' 'Harvard' 'Central' 'Kendall/MIT' 'Charles/MGH' 'Park Street'
 'South Station' 'Broadway' 'Andrew' 'JFK/UMass' 'Savin Hill'
 'Fields Corner' 'Shawmut' 'Ashmont' 'North Quincy' 'Wollaston'
 'Quincy Center' 'Quincy Adams' 'Braintree' 'Boston College'
 'South Street' 'Chestnut Hill Avenue' 'Chiswick Road' 'Sutherland Road'
 'Washington Street' 'Warren Street' 'Allston Street' 'Griggs Street'
 'Harvard Avenue' "Packard's Corner" nan 'Boston University Central'
 'Boston University East' 'Bla

In [10]:
for i in dfs:
    df = dfs[i]
    red_missing = [stop for stop in red_line if stop not in df['stop_name'].values]
    green_missing = [stop for stop in green_line if stop not in df['stop_name'].values]
    blue_missing = [stop for stop in blue_line if stop not in df['stop_name'].values]
    orange_missing = [stop for stop in orange_line if stop not in df['stop_name'].values]
    print(f"{i} Data")
    print("Red Line Missing Stops:", red_missing)
    print("Green Line Missing Stops:", green_missing)
    print("Blue Line Missing Stops:", blue_missing)
    print("Orange Line Missing Stops:", orange_missing)
    
    not_in_lines = df[~df['stop_name'].isin(red_line + green_line + blue_line + orange_line)]['stop_name'].unique()

    print("Stops not in any line arrays:", not_in_lines)

2016 Data
Red Line Missing Stops: ['Cedar Grove', 'Butler', 'Milton', 'Central Avenue', 'Valley Road', 'Capen Street', 'Mattapan']
Green Line Missing Stops: ['Medford/Tufts', 'Ball Square', 'Magoun Square', 'Gilman Square', 'East Somerville', 'Union Square', 'Lechmere', 'Amory Street', 'Babcock Street']
Blue Line Missing Stops: []
Orange Line Missing Stops: []
Stops not in any line arrays: [nan]
2017 Data
Red Line Missing Stops: ['Mattapan']
Green Line Missing Stops: ['Medford/Tufts', 'Ball Square', 'Magoun Square', 'Gilman Square', 'East Somerville', 'Union Square', 'Lechmere', 'Amory Street', 'Babcock Street']
Blue Line Missing Stops: []
Orange Line Missing Stops: []
Stops not in any line arrays: [nan]
2018 Data
Red Line Missing Stops: []
Green Line Missing Stops: ['Medford/Tufts', 'Ball Square', 'Magoun Square', 'Gilman Square', 'East Somerville', 'Union Square', 'Lechmere', 'Amory Street', 'Babcock Street']
Blue Line Missing Stops: []
Orange Line Missing Stops: []
Stops not in any 

Add a train line column to each row

In [11]:
def assign_line_dest(destination):
   if destination in red_line:
       return 'red'
   elif destination in green_line:
       return 'green'
   elif destination in blue_line:
       return 'blue'
   elif destination in orange_line:
       return 'orange'
   else:
       return 'unknown'

for i in dfs:
    df = dfs[i]
    df['line'] = df['destination'].apply(assign_line_dest)
    print(df.head())

   stop_id  direction_id   destination  headway_time_sec     stop_name  \
0    70001             1     Oak Grove        499.692239  Forest Hills   
1    70002             0  Forest Hills        509.656499  Green Street   
2    70003             1     Oak Grove        512.070998  Green Street   
3    70004             0  Forest Hills        506.538108   Stony Brook   
4    70005             1     Oak Grove        512.941418   Stony Brook   

                                   stop_desc    line  
0                 Forest Hills - Orange Line  orange  
1  Green Street - Orange Line - Forest Hills  orange  
2     Green Street - Orange Line - Oak Grove  orange  
3   Stony Brook - Orange Line - Forest Hills  orange  
4      Stony Brook - Orange Line - Oak Grove  orange  
   stop_id  direction_id   destination  headway_time_sec     stop_name  \
0    70001             1     Oak Grove        505.786136  Forest Hills   
1    70002             0  Forest Hills        505.573351  Green Street   
2  

In [12]:
def assign_line_from_description(row):
    if row['line'] == 'unknown':
        # Check if stop_desc is a string before calling lower()
        if isinstance(row['stop_desc'], str):
            desc_lower = row['stop_desc'].lower()
            if 'red' in desc_lower:
                return 'red'
            elif 'green' in desc_lower:
                return 'green'
            elif 'blue' in desc_lower:
                return 'blue'
            elif 'orange' in desc_lower:
                return 'orange'
    return row['line']

for i in dfs:
    df = dfs[i]
    df['line'] = df.apply(assign_line_from_description, axis=1)

Write the cleaned data to csv file

In [13]:
for i in dfs:
    df = dfs[i]
    df.to_csv(f"Data\Headways_{i}_cleaned.csv")

<>:3: SyntaxWarning: invalid escape sequence '\H'
<>:3: SyntaxWarning: invalid escape sequence '\H'
C:\Users\arosenolan\AppData\Local\Temp\ipykernel_25636\1401475552.py:3: SyntaxWarning: invalid escape sequence '\H'
  df.to_csv(f"Data\Headways_{i}_cleaned.csv")


Compile all csv files into one.

In [16]:
combined_dfs = []

for year in dfs:
    df = dfs[year]
    # Add a 'year' column to each DataFrame
    df['year'] = year
    combined_dfs.append(df)

# Concatenate all DataFrames vertically
combined_df = pd.concat(combined_dfs, ignore_index=True)

# Save to CSV
combined_df.to_csv('Headways_cleaned.csv', index=False)